# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Bring in and read weather data from weatherpy
weather_data = pd.read_csv('output_data/cities.csv')
weather_data.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Atuona,30,PF,1579875166,79,-9.80,-139.03,80.31,11.65
1,1,Albany,1,US,1579875166,85,42.60,-73.97,34.00,2.24
2,2,Tessalit,100,ML,1579875166,6,20.20,1.01,89.60,10.31
3,3,Sao Filipe,4,CV,1579875166,70,14.90,-24.50,74.17,14.85
4,4,Hilo,90,US,1579875166,83,19.73,-155.09,69.80,1.43


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
location = weather_data[['Lat', 'Lng']].astype(float)
humidity = weather_data["Humidity"].astype(float)


In [4]:
# Plot heatmap
fig = gmaps.figure()

# Creating the heatmap overlay
heat_layer = gmaps.heatmap_layer(location, weights = humidity, dissipating=False, max_intensity=humidity.max(), point_radius=1)

# Adding layer to map
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Ideal weather qualities (golf vacation): Between 70-80 degress, Wind under 10 mph, Humidity under 35%

In [5]:
# Creating the dataframe for the ideal Golf vacation
golf_df = weather_data[weather_data["Max Temp"] > 70]
golf_df = weather_data[weather_data["Max Temp"] < 80]
golf_df = weather_data[weather_data["Wind Speed"] < 10]
golf_df = weather_data[weather_data["Humidity"] < 35]
golf_df.count()

Unnamed: 0    33
City          33
Cloudiness    33
Country       31
Date          33
Humidity      33
Lat           33
Lng           33
Max Temp      33
Wind Speed    33
dtype: int64

In [6]:
# Clean Data
clean_golf_df = golf_df.dropna()
clean_golf_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
2,2,Tessalit,100,ML,1579875166,6,20.20,1.01,89.60,10.31
7,7,Viedma,11,AR,1579875167,21,-40.81,-63.00,75.11,9.57
8,8,Lebu,0,CL,1579875167,34,-37.62,-73.65,71.82,10.40
134,134,Bariloche,100,AR,1579875137,29,-41.15,-71.31,62.60,26.40
139,139,Nioro,95,GM,1579875183,11,13.35,-15.75,104.45,8.12


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = clean_golf_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

C:\Users\connor.roemelt\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
2,2,Tessalit,100,ML,1579875166,6,20.20,1.01,89.60,10.31,
7,7,Viedma,11,AR,1579875167,21,-40.81,-63.00,75.11,9.57,
8,8,Lebu,0,CL,1579875167,34,-37.62,-73.65,71.82,10.40,
134,134,Bariloche,100,AR,1579875137,29,-41.15,-71.31,62.60,26.40,
139,139,Nioro,95,GM,1579875183,11,13.35,-15.75,104.45,8.12,


In [8]:
# Settting parametters
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key    
}

for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f'{lat},{lng}'
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params = params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    

Missing field/result... skipping.


C:\Users\connor.roemelt\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [9]:
# It was indicated by the call in above that we need to drop some rows
clean_hotel_df = hotel_df
clean_hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
2,2,Tessalit,100,ML,1579875166,6,20.20,1.01,89.60,10.31,
7,7,Viedma,11,AR,1579875167,21,-40.81,-63.00,75.11,9.57,Hotel Austral Viedma
8,8,Lebu,0,CL,1579875167,34,-37.62,-73.65,71.82,10.40,Hotel Plaza Lebu
134,134,Bariloche,100,AR,1579875137,29,-41.15,-71.31,62.60,26.40,Huinid Bustillo Hotel & Spa
139,139,Nioro,95,GM,1579875183,11,13.35,-15.75,104.45,8.12,
148,148,Filingue,0,NE,1579875185,5,14.35,3.32,91.09,16.40,
163,163,Sangmelima,67,CM,1579875186,10,2.93,11.98,93.45,3.04,Hôtel L'Afamba
164,164,Yulara,57,AU,1579875186,21,-25.24,130.99,89.60,12.75,Sails in the Desert
190,190,Khudumelapye,56,BW,1579875189,23,-23.88,24.75,90.36,9.40,
193,193,Jimeta,55,NG,1579875189,5,9.28,12.47,93.38,9.89,AUN Hotel Conference Centre & Spa


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)
fig2 = gmaps.figure()
fig2.add_layer(marker_layer)
# Display Map
fig2

Figure(layout=FigureLayout(height='420px'))